In [52]:
import json
import sklearn
import numpy as np
import os
import pandas as pd
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [53]:
i = 1
cur_text = ''
all_texts = []
with open('Documents.csv', 'r') as f_in:
    lines = f_in.readlines() 

for line in lines:
    if line == '.Id %d\n' % i:
        all_texts.append(cur_text)
        cur_text = ''
        i += 1
    else:
        cur_text += line
all_texts.append(cur_text)

with open('queries.csv', 'r') as f_in:
    lines = f_in.readlines()

lines = lines[1:-1]
from_id_to_query = {
    int(line.split('\t', 1)[0]): line.split('\t', 1)[1] for line in lines
}

with open('relevance_test.csv', 'r') as f_in:
    lines = f_in.readlines()
lines = lines[1:]
queries = defaultdict(list)
for line in lines:
    line = line.strip()
    queries[int(line.split(',')[0])].append(int(line.split(',')[1]))

In [54]:
tfidf = TfidfVectorizer(min_df=1, stop_words='english', analyzer='word', ngram_range=(1,2))

In [55]:
tfidf.fit_transform(all_texts[1:])

<1400x94120 sparse matrix of type '<class 'numpy.float64'>'
	with 212855 stored elements in Compressed Sparse Row format>

In [56]:
ttov = [tfidf.transform([some_text]) for some_text in all_texts]

In [57]:
ttov[0]

<1x94120 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [59]:
with open('results.csv', 'w') as f_out:
    f_out.write('QueryId,DocumentId\n')
    for k, v in queries.items():
        if k not in from_id_to_query:
            print(k)
            for _v in v:
                f_out.write(str(k) + ',' + str(_v) + '\n')
            continue
        k_vec = tfidf.transform([from_id_to_query[k]]).toarray()
        v = sorted(v, key=lambda x: cosine_similarity(ttov[x], k_vec), reverse=True)
        for _v in v:
                f_out.write(str(k) + ',' + str(_v) + '\n')
        

127
129
134
144
151
159
162
166
172
174
178
179
180
185
186
188
191
192
193
194
195
197
198
199
207
220
221
222


Private Score 0.54791
Public Score 0.52008